In [1]:

model_name="AXCXEPT/EZO-Common-9B-gemma-2-it"
# トークナイザーとモデルの準備
from unsloth import FastLanguageModel
max_seq_length = 8192


/home/setup/miniconda3/envs/unsloth_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, # Supports Llama, Mistral - replace this!
    max_seq_length = max_seq_length, # Supports RoPE Scaling internally, so choose any!
    load_in_4bit = False,
)

Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.9: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.15 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.11it/s]


In [3]:
model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
 

In [4]:
import torch

from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

# Get dataset
dataset_name="kanhatakeyama/0717-calm3-22b-random-genre-inst-sft-tsub-part"
dataset = load_dataset(dataset_name, split = "train")


In [5]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!

        chat = [
            { "role": "user", "content": instruction },
            { "role": "assistant", "content": output},
        ]
        text= tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)+EOS_TOKEN
        #text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset[0]

{'genre': 'ニュース記事作成:,広告スパム検出:広告目的のスパムの識別,ユーザーの要望に基づいたレポートをカスタマイズする',
 'instruction': '問題文を一つのみ出力します。\n\n---\n\n広告を目的としたスパムを識別するために、ユーザーの要望に基づいたニュース記事をどのようにカスタマイズしますか？特に、広告が誤解を招く内容を含んでいないことを保証しつつ、信頼性のある情報源に基づいた内容をどのように選択し、ユーザーにとって有益なコンテンツを提供するアプローチについて説明してください。',
 'output': 'ユーザーの要望に基づいたニュース記事をカスタマイズするためには、まずユーザーの興味や関心を深く理解することが重要です。これには、ユーザーのプロフィール情報や過去の行動履歴、検索履歴などを分析することで、どのようなトピックに関心があるのかを把握します。次に、信頼性のある情報源を選定するプロセスが必要です。これは、ニュース記事の信頼性を確保するために、定評のあるメディアや専門機関からの情報を中心に収集することを意味します。\n\n具体的には、以下のようなステップを踏むことが考えられます。\n\n1. ユーザーの興味に基づくトピックの選定:\n   - ユーザーが過去に閲覧した記事や検索したキーワードを基に、関連するトピックを特定します。\n   - ソーシャルメディアでの発言やコメントも参考にし、ユーザーが関心を持っているテーマを深掘りします。\n\n2. 信頼性のある情報源の特定:\n   - ニュース記事を収集する際には、ファクトチェックを行っているメディアや、ジャーナリズムの倫理基準に準拠している機関を優先します。\n   - 例えば、BBC、The New York Times、The Guardianなど、国際的に評価の高いニュースソースを利用します。\n\n3. 広告のフィルタリングと検証:\n   - 広告が誤解を招く内容を含んでいないことを確認するために、広告主の背景や過去の実績を調査します。\n   - 広告の内容が事実に基づいているかどうかを確認するために、複数の情報源を比較検討します。\n\n4. ユーザーにとって有益なコンテンツの提供:\n   - 選定したニュース記事の中から、ユーザーの興味に最も合致する

In [7]:

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
      per_device_train_batch_size = 2,
      #gradient_accumulation_steps = 4,
      warmup_steps = 10,
      max_steps = 60,
      fp16 = not torch.cuda.is_bf16_supported(),
      bf16 = torch.cuda.is_bf16_supported(),
      logging_steps = 1,
      output_dir = "outputs",
      optim = "adamw_8bit",
      seed = 3407,
  ),
)
trainer.train()

Map: 100%|██████████| 9096/9096 [00:02<00:00, 4513.81 examples/s]
max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,096 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 1
\        /    Total batch size = 2 | Total steps = 60
 "-____-"     Number of trainable parameters = 0


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [8]:
model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
 